In [ ]:
import pandas as pd

desired_columns = ['steam_appid', 'name', 'developers', 'publishers', 'categories', 'genres', 'required_age', 'n_achievements', 'platforms',
                   'total_reviews', 'total_positive', 'total_negative', 'review_score', 'review_score_desc', 'positive_percentual', 'metacritic', 'is_free', 'price_initial (USD)']

df = pd.read_csv('steam_games.csv', usecols=desired_columns)

filtered_df = df[df['total_reviews'] > 5000]
print(filtered_df.size)
filtered_df.head()

13068


,steam_appid,name,developers,publishers,categories,genres,required_age,n_achievements,platforms,total_reviews,total_positive,total_negative,review_score,review_score_desc,positive_percentual,metacritic,is_free,price_initial (USD)
4804,2567870,Chained Together,['Anegar Games'],['Anegar Games'],"['Single-player', 'Multi-player', 'Co-op', 'On...","['Adventure', 'Casual', 'Indie', 'Simulation']",0,16,['windows'],16138,14865,1273,8.0,Very Positive,92.1,0,False,4.99
6496,2519060,Call of Duty®: Modern Warfare® III,"['Sledgehammer Games', 'Treyarch', 'Infinity W...",['Activision'],"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],0,0,['windows'],6784,2015,4769,4.0,Mostly Negative,29.7,0,False,69.99
7163,1063660,Bendy and the Dark Revival,['Joey Drew Studios'],['Joey Drew Studios'],"['Single-player', 'Steam Achievements', 'Full ...","['Action', 'Adventure']",72,38,['windows'],6492,6038,454,8.0,Very Positive,93.0,72,False,29.99
7653,1049410,Superliminal,['Pillow Castle'],['Pillow Castle'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",0,27,"['windows', 'mac', 'linux']",12615,11840,775,8.0,Very Positive,93.9,0,False,19.99
7744,1046030,ISLANDERS,"['Coatsink', 'GrizzlyGames', 'Stage Clear Stud...",['Coatsink'],"['Single-player', 'Steam Achievements', 'Steam...","['Casual', 'Indie', 'Strategy']",82,38,"['windows', 'mac']",9886,9414,472,9.0,Overwhelmingly Positive,95.2,82,False,4.99


In [38]:
import requests

data = {'steam_appid': [], 'current_player_count': []}
for index, row in filtered_df.iterrows():
    app_id = row['steam_appid']
    url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?key=29C480E49AD963D9F69D9FCCA9A4F2A1&steamids=76561197960435530&appid=" + str(app_id)
    players = requests.get(url).json()["response"]["player_count"]

    data['steam_appid'].append(app_id)
    data['current_player_count'].append(players)

player_count_df = pd.DataFrame(data)
player_count_df.to_csv("current_players.csv", index=False)
    

In [43]:
data = {'steam_appid': [], 'release_date': []}
for index, row in filtered_df.iterrows():
    app_id = row['steam_appid']
    url = "https://store.steampowered.com/api/appdetails?appids=" + str(app_id)

    info = requests.get(url).json()

    if info is not None:
        release_date = info[str(app_id)]["data"]["release_date"]["date"]
    else:
        release_date = 'N/A'

    data['steam_appid'].append(app_id)
    data['release_date'].append(release_date)

release_date_df = pd.DataFrame(data)
release_date_df.to_csv("games_release_date.csv", index=False)

In [44]:
merged_df1 = pd.merge(filtered_df, player_count_df, on='steam_appid', how='inner')
merged_df2 = pd.merge(merged_df1, release_date_df, on='steam_appid', how='inner')

# Save the combined DataFrame to a new CSV file
merged_df2.to_csv('games.csv', index=False)

merged_df2.head()

,steam_appid,name,developers,publishers,categories,genres,required_age,n_achievements,platforms,total_reviews,total_positive,total_negative,review_score,review_score_desc,positive_percentual,metacritic,is_free,price_initial (USD),current_player_count,release_date
0,2567870,Chained Together,['Anegar Games'],['Anegar Games'],"['Single-player', 'Multi-player', 'Co-op', 'On...","['Adventure', 'Casual', 'Indie', 'Simulation']",0,16,['windows'],16138,14865,1273,8.0,Very Positive,92.1,0,False,4.99,909,"19 Jun, 2024"
1,2519060,Call of Duty®: Modern Warfare® III,"['Sledgehammer Games', 'Treyarch', 'Infinity W...",['Activision'],"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],0,0,['windows'],6784,2015,4769,4.0,Mostly Negative,29.7,0,False,69.99,0,N/A
2,1063660,Bendy and the Dark Revival,['Joey Drew Studios'],['Joey Drew Studios'],"['Single-player', 'Steam Achievements', 'Full ...","['Action', 'Adventure']",72,38,['windows'],6492,6038,454,8.0,Very Positive,93.0,72,False,29.99,38,N/A
3,1049410,Superliminal,['Pillow Castle'],['Pillow Castle'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",0,27,"['windows', 'mac', 'linux']",12615,11840,775,8.0,Very Positive,93.9,0,False,19.99,309,N/A
4,1046030,ISLANDERS,"['Coatsink', 'GrizzlyGames', 'Stage Clear Stud...",['Coatsink'],"['Single-player', 'Steam Achievements', 'Steam...","['Casual', 'Indie', 'Strategy']",82,38,"['windows', 'mac']",9886,9414,472,9.0,Overwhelmingly Positive,95.2,82,False,4.99,23,"4 Apr, 2019"
